# Google Drive 연결

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 라이브러리

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import (
    roc_auc_score, confusion_matrix, accuracy_score,
    precision_score, recall_score, f1_score, classification_report
)

# 데이터 가져오기

In [ ]:
DATA_PATH = '/content/drive/MyDrive/Colab Notebooks/Multicampus-8/4_머신러닝_딥러닝/프로젝트/'

train_df = pd.read_csv(DATA_PATH + "dataset/train.csv")
test_df = pd.read_csv(DATA_PATH + "dataset/test.csv")
target = "diagnosed_diabetes"

# 데이터 분리

In [ ]:
X = train_df[['age', 'physical_activity_minutes_per_week', 'diet_score','bmi','waist_to_hip_ratio' ,'systolic_bp','triglycerides','hdl_cholesterol','family_history_diabetes', 'hypertension_history']]
y = train_df[target]
test = test_df[['age', 'physical_activity_minutes_per_week', 'diet_score','bmi' ,'waist_to_hip_ratio' ,'systolic_bp','triglycerides','hdl_cholesterol','family_history_diabetes', 'hypertension_history']]


In [ ]:
# X['age_group'] = pd.cut(
#     X['age'],
#     bins=[18, 42, 50, 58, 90],   # 경계값
#     labels=['young', 'middle', 'senior', 'elder'],
#     right=False
# )
# test['age_group'] = pd.cut(
#     test['age'],
#     bins=[18, 42, 50, 58, 90],
#     labels=['young', 'middle', 'senior', 'elder'],
#     right=False
# )

In [ ]:
# BMI × WHR 조합 변수
X['bmi_whr'] = X['bmi'] * X['waist_to_hip_ratio']
test['bmi_whr'] = test['bmi'] * test['waist_to_hip_ratio']
X =X.drop('waist_to_hip_ratio',axis=1)
test =test.drop('waist_to_hip_ratio',axis=1)

/tmp/ipython-input-1540205429.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['bmi_whr'] = X['bmi'] * X['waist_to_hip_ratio']
/tmp/ipython-input-1540205429.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['bmi_whr'] = test['bmi'] * test['waist_to_hip_ratio']


In [ ]:
X['bmi_age'] = X['bmi'] * X['age']
test['bmi_whr'] = test['bmi'] * test['age']


In [ ]:
X_train_raw, X_val_raw, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)


# 전처리 설계

In [ ]:
from sklearn.preprocessing import FunctionTransformer, PowerTransformer, PolynomialFeatures
from sklearn.pipeline import FeatureUnion

# # # Ordinal Encoding
# ordinal_features = ['age_group']
# ordinal_categories = [['young', 'middle', 'senior', 'elder']]  # 순서 명시

# # OneHot Encoding 컬럼들
categorical_features = X_train_raw.select_dtypes(include=['object']).columns.tolist()

# 순서형 자료 컬럼은 제거
# 집합 연산
categorical_features = list(set(categorical_features))

numeric_features = X_train_raw.select_dtypes(include=np.number).columns.tolist()
removes = X_train_raw[['family_history_diabetes', 'hypertension_history']].columns.tolist()
numeric_features = list(set(numeric_features)- set(removes))


# [3] 전처리기 업데이트
# triglycerides는 로그 변환 후 스케일링, 나머지는 표준화만 진행하도록 세분화 가능합니다.
preprocessor = ColumnTransformer(
    transformers=[
        # 중성지방: 로그변환(또는 PowerTransformer) 후 스케일링
        ("trig_log", Pipeline([
            ("power", PowerTransformer(method='yeo-johnson')), # Skewness 자동 보정
            ("scaler", StandardScaler())
        ]), ['triglycerides']),

        # # 혈압 및 주요지표: 제곱항 추가 후 스케일링
        # ("bp_poly", Pipeline([
        #     ("poly", PolynomialFeatures(degree=2, include_bias=False)),
        #     ("scaler", StandardScaler())
        # ]), ['bmi_whr']),

        # 나머지 수치형
        ("num", StandardScaler(), list(set(numeric_features) - {'triglycerides', 'bmi_whr'})),
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), categorical_features)
        # ("ord", OrdinalEncoder(categories=ordinal_categories), ordinal_features)
    ],
    remainder="passthrough"
)

# 모델 개발 및 하이퍼파라미터 설정
- 랜덤서치는 확률분포나 범위 지정 가능

1. 클래스 불균형 처리를 위한 비율 계산 (`ratio`)
- **의미**: 학습 데이터(`y_train`) 내 다수 클래스(0: 정상) 대비 소수 클래스(1: 당뇨)의 비율을 산출합니다.
- **용도**: 데이터 불균형이 심할 때, 모델이 소수 클래스에 더 가중치를 두어 학습하도록 돕는 지표로 활용됩니다.



2. 로지스틱 회귀(Logistic Regression) 설정
- **의미**: 선형 분류 모델을 기반으로 하이퍼파라미터 탐색 범위를 정의합니다.
- **`class_weight="balanced"`**: 데이터 빈도에 반비례하여 가중치를 자동 부여하여 불균형을 해결합니다.
- **`model__C`**: 규제(Regularization) 강도의 역수입니다. 값이 작을수록 더 강력한 규제를 적용합니다.

3. 결정 트리(Decision Tree) 설정
- **의미**: 데이터 규칙을 나무 구조로 시각화하는 비선형 모델의 탐색 공간을 정의합니다.
- **`model__max_depth`**: 나무의 최대 깊이를 제한하여 모델이 너무 복잡해지는 것(과적합)을 방지합니다.
- **`model__min_samples_leaf`**: 리프 노드(말단)가 되기 위한 최소 샘플 수로, 가지치기의 역할을 수행합니다.



4. XGBoost 설정
- **의미**: 성능이 우수한 그래디언트 부스팅(GBDT) 알고리즘의 최적 파라미터를 탐색합니다.
- **`scale_pos_weight=ratio`**: 앞서 계산한 클래스 비율을 적용하여 양성 클래스(1)에 대한 가중치를 직접 조절합니다.
- **`model__learning_rate`**: 각 단계에서 오차를 보정하는 보폭을 결정하며, `max_depth` 및 `n_estimators`와 결합하여 성능을 최적화합니다.
- **`model__subsample`**: 나무를 만들 때 사용하는 무작위 샘플링 비율로, 모델의 일반화 성능을 높입니다.

In [ ]:
ratio = (y_train == 0).sum() / (y_train == 1).sum()
print(ratio)


model_configs = [
    {
        "name": "Logistic Regression",
        "model": LogisticRegression(max_iter=1000, class_weight="balanced", random_state=42),
        "params": {
            "model__C": [0.001, 0.01, 0.1, 1.0, 10.0, 100.0],
            "model__penalty": ['l2']
        }
    },
    {
        "name": "XGBoost",
        # device="cuda" 추가 (최신 버전 기준)
        "model": XGBClassifier(
            eval_metric="logloss",
            random_state=42,
            tree_method='hist', # GPU 최적화 히스토그램 방식
            device="cuda",      # GPU 사용 설정
            scale_pos_weight=ratio
        ),
        "params": {
            "model__n_estimators": [100, 200, 300, 400],
            "model__learning_rate": [0.01, 0.05, 0.1, 0.2],
            "model__max_depth": [3, 4, 5, 6],
            "model__subsample": [0.7, 0.8, 0.9]
        }
    },
    {
        "name": "LightGBM",
        # device="gpu" 추가
        "model": LGBMClassifier(
            random_state=42,
            device="gpu",       # GPU 사용 설정
            gpu_platform_id=0,
            gpu_device_id=0,
            scale_pos_weight=ratio,
            verbosity=-1
        ),
        "params": {
            "model__n_estimators": [100, 200, 300, 400],
            "model__learning_rate": [0.01, 0.05, 0.1, 0.2],
            "model__subsample": [0.7, 0.8, 0.9],
            "model__max_depth": [3, 4, 5, 6]
        }
    }
]

0.6043730625762793


# Random Search 학습 진행

In [ ]:
import time
from datetime import timedelta
import warnings
from tqdm.auto import tqdm # 진행 바 라이브러리

warnings.filterwarnings('ignore', category=UserWarning)

# 1. 교차 검증 및 결과 저장 준비
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
results = []

# 2. 전체 모델 루프에 tqdm 적용 (전체 진행률 확인)
for config in tqdm(model_configs, desc="전체 모델 최적화 진행 중"):
    print(f"\n [{config['name']}] 랜덤 서치 탐색을 시작합니다...")

    # 시간 측정 시작
    start_time = time.perf_counter()

    pipeline = Pipeline([("prep", preprocessor), ("model", config['model'])])

    # RandomizedSearchCV 설정
    # verbose=1: 간단한 진행 상황 출력 (각 파라미터 조합 시도 시 출력)
    # n_jobs=-1: 병렬 처리 활성화
    search = RandomizedSearchCV(
        pipeline,
        param_distributions=config['params'],
        n_iter=10,
        cv=cv,
        scoring='roc_auc',
        n_jobs=-1,
        random_state=42,
        verbose=1 # <--- 실행 중인 상태를 로그로 보여줍니다.
    )

    # 모델 학습 (실제 탐색 수행)
    search.fit(X_train_raw, y_train)

    # 시간 측정 종료 및 변환
    end_time = time.perf_counter()
    elapsed_time = end_time - start_time
    elapsed_hms = str(timedelta(seconds=int(elapsed_time)))

    # 검증셋 평가
    val_probs = search.predict_proba(X_val_raw)[:, 1]
    val_auc = roc_auc_score(y_val, val_probs)
    val_pred = search.predict(X_val_raw)

    # 상세 지표 계산
    tn, fp, fn, tp = confusion_matrix(y_val, val_pred).ravel()

    print(f"[{config['name']}] 탐색 완료!")
    print(f"소요 시간: {elapsed_hms} | Best CV AUC: {search.best_score_:.4f} | Val AUC: {val_auc:.4f}")

    # 결과 저장
    results.append({
        "Model": config['name'],
        "AUC": roc_auc_score(y_val, val_probs),
        "Accuracy": accuracy_score(y_val, val_pred),
        "Precision": precision_score(y_val, val_pred),
        "Recall": recall_score(y_val, val_pred),
        "F1": f1_score(y_val, val_pred),
        "TP": tp, "TN": tn, "FP": fp, "FN": fn,
        "Best_CV_AUC": search.best_score_,
        "Val_AUC": val_auc,
        "Time": elapsed_hms,
        "Best_Params": search.best_params_
    })

전체 모델 최적화 진행 중:   0%|          | 0/3 [00:00<?, ?it/s]


 [Logistic Regression] 랜덤 서치 탐색을 시작합니다...
Fitting 5 folds for each of 6 candidates, totalling 30 fits
[Logistic Regression] 탐색 완료!
소요 시간: 0:00:50 | Best CV AUC: 0.6926 | Val AUC: 0.6933

 [XGBoost] 랜덤 서치 탐색을 시작합니다...
Fitting 5 folds for each of 10 candidates, totalling 50 fits
[XGBoost] 탐색 완료!
소요 시간: 0:01:29 | Best CV AUC: 0.7222 | Val AUC: 0.7227

 [LightGBM] 랜덤 서치 탐색을 시작합니다...
Fitting 5 folds for each of 10 candidates, totalling 50 fits
[LightGBM] 탐색 완료!
소요 시간: 0:07:11 | Best CV AUC: 0.7238 | Val AUC: 0.7243


# 결과 확인

In [ ]:
# 결과 확인
import pandas as pd
results_df = pd.DataFrame(results)
results_df

,Model,AUC,Accuracy,Precision,Recall,F1,TP,TN,FP,FN,Best_CV_AUC,Val_AUC,Time,Best_Params
0,Logistic Regression,0.693295,0.625964,0.753863,0.593770,0.664308,51813,35822,16917,35448,0.692552,0.693295,0:00:50,"{'model__penalty': 'l2', 'model__C': 1.0}"
1,XGBoost,0.722742,0.653379,0.772154,0.629697,0.693687,54948,36525,16214,32313,0.722172,0.722742,0:01:29,"{'model__subsample': 0.9, 'model__n_estimators..."
2,LightGBM,0.724286,0.653921,0.773896,0.628333,0.693560,54829,36720,16019,32432,0.723765,0.724286,0:07:11,"{'model__subsample': 0.9, 'model__n_estimators..."


In [ ]:
display(results_df[["Model", "Best_CV_AUC", "Val_AUC", "Time"]])

,Model,Best_CV_AUC,Val_AUC,Time
0,Logistic Regression,0.692552,0.693295,0:00:50
1,XGBoost,0.722172,0.722742,0:01:29
2,LightGBM,0.723765,0.724286,0:07:11


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 성능(AUC)과 시간(초 단위) 비교 차트
results_df['Seconds'] = results_df['Time'].apply(lambda x: sum(int(a) * 60**i for i, a in enumerate(reversed(x.split(':')))))

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Bar(x=results_df['Model'], y=results_df['Val_AUC'], name="Validation AUC"), secondary_y=False)
fig.add_trace(go.Scatter(x=results_df['Model'], y=results_df['Seconds'], name="Time (seconds)", mode='lines+markers'), secondary_y=True)

fig.update_layout(title_text="모델별 성능 및 최적화 소요 시간 비교")
fig.update_yaxes(title_text="AUC Score", secondary_y=False)
fig.update_yaxes(title_text="Time (s)", secondary_y=True)
fig.show()

# 제출 파일 생성코드

In [ ]:
# 1. 랜덤 서치 결과 중 가장 성능(Val_AUC)이 좋은 모델 정보 가져오기
best_model_info = max(results, key=lambda x: x['Val_AUC'])
best_model_name = best_model_info['Model']
best_params = best_model_info['Best_Params']

print(f"🏆 최종 선택된 모델: {best_model_name}")
print(f"⚙️ 최적 하이퍼파라미터: {best_params}")

# 2. 최적 모델 객체 찾기 및 파라미터 적용
# model_configs 리스트에서 이름이 같은 모델 설정을 찾습니다.
selected_config = next(config for config in model_configs if config["name"] == best_model_name)
final_model_obj = selected_config["model"]

# 파생된 파이프라인 생성 (전처리기 + 선택된 모델)
final_pipeline = Pipeline([
    ("prep", preprocessor),
    ("model", final_model_obj)
])

# RandomizedSearchCV에서 찾은 'best_params'를 파이프라인에 직접 주입합니다.
final_pipeline.set_params(**best_params)

# 3. 전체 데이터로 재학습 (Train + Validation 합쳐서 학습하면 성능이 더 좋아집니다)
# 강의용이라면 간단하게 X_train_raw만 사용해도 무방합니다.
print("최적 파라미터로 모델 재학습 중...")
final_pipeline.fit(X_train_raw, y_train)

# 4. 테스트 데이터 예측
test_X = test_df.drop(["id"], axis=1, errors="ignore")
test_proba = final_pipeline.predict_proba(test_X)[:, 1]

# 5. 제출 데이터프레임 구성
submission = pd.DataFrame({
    "id": test_df["id"] if "id" in test_df.columns else np.arange(len(test_df)),
    "diagnosed_diabetes": test_proba
})

# 6. 파일 저장 및 확인
file_name = f"submission_best_{best_model_name.lower().replace(' ', '_')}.csv"
save_path = DATA_PATH + "output/" + file_name
submission.to_csv(save_path, index=False)

print(f"제출 파일 저장 완료: {save_path}")
display(submission.head())

In [ ]:
# 5. 파일 다운로드 (Colab)
from google.colab import files
files.download(save_path)